In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()
sns.set_context('talk')

import warnings
warnings.filterwarnings('ignore')

import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

In [9]:
df = pd.read_csv("373594600_122019_1339_airline_delay_causes.csv")

In [10]:
df2 = pd.read_csv("runways.csv")

In [11]:
df_city = pd.read_csv("cities_need.csv")
df_city.columns = ['ABBR','municipality']
df_single = pd.DataFrame([['ABE','allentown']],columns = ['ABBR','municipality'])
df_city = pd.concat([df_single, df_city]).reset_index(drop = True) 
df_city.to_csv('cities_needed.csv')

In [12]:
df3 = pd.read_csv("airports.csv")
df3 = df3[df3['iso_country'] == "US"]

In [13]:
for i in df3['municipality'].keys():
    df3['municipality'][i] = str(df3['municipality'][i]).lower()

In [14]:
df2

,id,airport_ref,airport_ident,length_ft,width_ft,surface,lighted,closed,le_ident,le_latitude_deg,le_longitude_deg,le_elevation_ft,le_heading_degT,le_displaced_threshold_ft,he_ident,he_latitude_deg,he_longitude_deg,he_elevation_ft,he_heading_degT,he_displaced_threshold_ft
0,269408,6523,00A,80.0,80.0,ASPH-G,1,0,H1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,255155,6524,00AK,2500.0,70.0,GRVL,0,0,N,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN
2,254165,6525,00AL,2300.0,200.0,TURF,0,0,01,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN
3,270932,6526,00AR,40.0,40.0,GRASS,0,0,H1,NaN,NaN,NaN,NaN,NaN,H1,NaN,NaN,NaN,NaN,NaN
4,322128,322127,00AS,1450.0,60.0,Turf,0,0,1,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41462,327703,327451,ZYSQ,8202.0,148.0,NaN,0,0,04,NaN,NaN,NaN,NaN,NaN,22,NaN,NaN,NaN,NaN,NaN
41463,235188,27242,ZYTL,10827.0,148.0,CON,1,0,10,38.9671,121.520,105.0,95.6,650.0,28,38.9642,121.558,85.0,275.6,320.0
41464,235186,27243,ZYTX,10499.0,148.0,CON,1,0,06,41.6304,123.469,171.0,48.7,NaN,24,41.6493,123.498,197.0,228.7,NaN
41465,235169,27244,ZYYJ,8530.0,148.0,CON,1,0,09,42.8811,129.436,623.0,81.7,NaN,27,42.8845,129.467,597.0,261.7,NaN


In [15]:
df_city_with_ident = pd.merge(df3,df_city,how = 'inner', on = 'municipality')
df_city_with_ident.shape

(3513, 19)

In [16]:

df2 = df2.rename(columns={'airport_ident':'ident'})
df_runway = pd.merge(df2,df_city_with_ident, how = 'inner', on = 'ident')
df_runway = pd.merge(df_city,df_runway,on = 'municipality')
RW_MEAN = df_runway.groupby(['ABBR_x']).mean()
RW_SUM = df_runway.groupby(['ABBR_x']).sum()
RW_SUM = RW_SUM[['length_ft','width_ft']]
RW_MEAN = RW_MEAN[['length_ft','width_ft']]

RW = RW_SUM['length_ft'].to_frame().join(RW_MEAN['width_ft'].to_frame())

In [17]:
RW_MEAN.to_csv('runway_mean.csv')

In [18]:
RW_SUM.to_csv('runway_sum.csv')

In [19]:
RW.to_csv('runway_sumL_meanW.csv')

In [20]:
df_NV = pd.read_csv("navaids.csv")

In [21]:
df_NV = df_NV[df_NV['iso_country'] == 'US']
df_NV

,id,filename,ident,name,type,frequency_khz,latitude_deg,longitude_deg,elevation_ft,iso_country,dme_frequency_khz,dme_channel,dme_latitude_deg,dme_longitude_deg,dme_elevation_ft,slaved_variation_deg,magnetic_variation_deg,usageType,power,associated_airport
125,85178,Kenie_NDB_US,AA,Kenie,NDB,365,47.009300,-96.815201,887.0,US,NaN,NaN,NaN,NaN,NaN,NaN,4.089,TERMINAL,MEDIUM,KFAR
126,85179,Cedar_NDB_US,AA,Cedar,NDB,341,33.533001,-82.614403,517.0,US,NaN,NaN,NaN,NaN,NaN,NaN,-5.217,TERMINAL,LOW,NaN
127,85180,Abraham_NDB_US,AAA,Abraham,NDB,329,40.160099,-89.337799,593.0,US,NaN,NaN,NaN,NaN,NaN,NaN,-1.206,TERMINAL,LOW,NaN
129,85182,Apalachicola_NDB_US,AAF,Apalachicola,NDB,349,29.723301,-85.028099,19.0,US,NaN,NaN,NaN,NaN,NaN,NaN,-2.557,TERMINAL,LOW,NaN
137,85190,Ashland_NDB_US,AAU,Ashland,NDB,329,40.963299,-82.253403,1256.0,US,NaN,NaN,NaN,NaN,NaN,NaN,-7.232,TERMINAL,LOW,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10680,95765,Youngstown_VORTAC_US,YNG,Youngstown,VORTAC,109000,41.331001,-80.674698,1160.0,US,109000.0,027X,NaN,NaN,NaN,-5.001,-8.447,BOTH,MEDIUM,KYNG
10736,95824,York_VORTAC_US,YRK,York,VORTAC,112800,38.644100,-82.978302,1040.0,US,112800.0,075X,NaN,NaN,NaN,-5.001,-6.125,LO,MEDIUM,NaN
10901,95999,Zephyr_NDB_US,ZEF,Zephyr,NDB,326,36.313202,-80.723396,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN,-7.221,TERMINAL,LOW,NaN
11033,96135,Zuni_VORTAC_US,ZUN,Zuni,VORTAC,113400,34.965801,-109.154999,6550.0,US,113400.0,081X,NaN,NaN,NaN,14.001,10.490,BOTH,HIGH,NaN


In [22]:
df_air = pd.read_csv('Airports_perfect.csv')

In [23]:
for i in df_air['City'].keys():
    df_air['City'][i] = str(df_air['City'][i]).lower()
df_air['City'].value_counts()

houston         108
wasilla          63
los angeles      52
columbus         47
jackson          42
               ... 
waskom            1
cocodrie          1
camarillo         1
pritchett         1
lehigh acres      1
Name: City, Length: 8135, dtype: int64

In [24]:
df_air = df_air.rename(columns = {'City':'municipality'})
df_air[df_air['Site_Num'] == 22]
#df_one = pd.merge(df_air,df_city,how = 'inner', on = 'municipality')
#df_Inspection = df_one['Last_Inspection'].dropna()
#df_time = df_one[['Activation_Date','municipality']].groupby('municipality')


,X,Y,OBJECTID,Rec_Type,Site_Num,Fac_Type,Loc_Id,Eff_Date,Region_Code,Field_Office_Code,...,Icao_Identifier,Min_Op_Net,Filler,origin,Enplanements,Passengers,Arrivals,Departures,Freightlbs,Maillbs


In [92]:
df_A = pd.read_csv("airports-extended.csv")

In [91]:
df_A = df_A[['Goroka Airport','Goroka','Papua New Guinea', 'GKA','-6.081689834590001','145.391998291']]
df_A = df_A.rename(columns = {'Goroka Airport':'Facility Name','Goroka':'City','Papua New Guinea':'country','GKA':'ident'})
df_A = df_A.rename(columns = {'-6.081689834590001' : 'Lat','145.391998291':'Long'})
df_sub = pd.DataFrame({'Facility Name':['Goroka Airport'],'City':['Goroka'],'country':['Papua New Guinea'],'ident':['GKA'],'Lat':[-6.081689834590001],'Long':[145.391998291]})
df_sub.append(df_A,ignore_index = True)
df_sub.to_csv('airport_processed.csv')

In [ ]:
d = {'Facility Name': 'Goroka Airport','City': 'Goroka', 'Country':Papua }